In [ ]:
def marginal_likelihood(t, y, sigma=0, kernel=ExpSinSquaredKernel, **kwargs):
    """
    
    """
    # The covariance and its determinant
    npts = len(t)
    K = kernel(t, **kwargs) + sigma ** 2 * np.eye(npts)
    detK = np.linalg.det(K)
    
    # The marginal likelihood
    log_like = -0.5 * np.dot(y.T, np.linalg.solve(K, y))
    log_like -= -0.5 * np.log(detK)
    log_like -= 0.5 * npts * np.log(2 * np.pi)
    
    return log_like

In [ ]:
# TODO: Dfm's fitting a line example.